In [ ]:
ROOT_PATH = '/content/drive/MyDrive/btp'

def _path(path):
  return f'{ROOT_PATH}/{path}'

In [ ]:
!pip install -q pytorch-lightning 
!pip install -q bpemb  --no-deps
!pip install -q sentencepiece
!pip install -q transformers

     |████████████████████████████████| 527 kB 7.8 MB/s 
     |████████████████████████████████| 829 kB 48.0 MB/s 
     |████████████████████████████████| 952 kB 46.2 MB/s 
     |████████████████████████████████| 396 kB 60.2 MB/s 
     |████████████████████████████████| 133 kB 52.6 MB/s 
     |████████████████████████████████| 596 kB 57.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.9 MB/s 
     |████████████████████████████████| 271 kB 54.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 48.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 1.2 MB 10.3 MB/s 
     |████████████████████████████████| 3.5 MB 7.6 MB/s 
     |████████████████████████████████| 6

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import get_scheduler, AdamW

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


# import torchtext
# from torchtext import datasets, vocab
# from torchtext.legacy import data as textdata
## ???from torchtext.vocab import GloVe

import numpy as np
import pandas as pd

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

from bpemb import BPEmb

import random
import os

from argparse import ArgumentParser

import pickle
from collections import namedtuple 
Transition = namedtuple('Transition',['entity', 'property', 'value'])



In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [ ]:
def parse_args():
	parser = ArgumentParser(description='Word Meaning Comparison')
	parser.add_argument('--data', '-d', type=str, default='./sampled.pkl')
	parser.add_argument('--embedding', '-e', type=str, default='bpe', choices=['bpe'])

	parser.add_argument('--gpu', type=int, default=1)
	parser.add_argument('--batch_size', type=int, default=32)
	parser.add_argument('--epochs', type=int, default=25)
	parser.add_argument('--lr', type=float, default=0.001)
	parser.add_argument('--lr_decay', type=float, default=0.0005)	# removed

	parser.add_argument('--rnn_hidden_dim', type=int, default=16)
	parser.add_argument('--num_rnn_layers', type=int, default=1)
	parser.add_argument('--dropout', type=float, default=0.5)

	return parser.parse_known_args()[0]


args = parse_args()
print(args)

Namespace(batch_size=32, data='./sampled.pkl', dropout=0.5, embedding='bpe', epochs=25, gpu=1, lr=0.001, lr_decay=0.0005, num_rnn_layers=1, rnn_hidden_dim=16)


In [ ]:
# # open 'processed_data/sampled.pkl'
with open(args.data, 'rb') as f:
    data = pickle.load(f)

In [ ]:
# build Dataset class
class TechTrackDataset(Dataset):
    def __init__(self, data, embedding='bpe'):
        assert len(data) > 0, 'data must be a non-empty list'

        assert embedding in ['bpe'], 'embedding must be bpe'
        self.embedding_name = embedding
        if self.embedding_name == 'bpe':
            PRETRAINED_EMBEDDING_DIM = 50
            self.EMBEDDING_DIM = 2*PRETRAINED_EMBEDDING_DIM + 1
            self.embedding = BPEmb(lang='en', dim=PRETRAINED_EMBEDDING_DIM, vs=200000)

        self._PROPERTIES_MAP = {
            'isConnected': 'connected',
            'isOpened': 'opened',
            'isPowered': 'powered',
            'isInstalled': 'installed',
            'isSettingsChanged': 'settings changed',
            'isUsed': 'used',
            'isSetup': 'setup',
            'isRelatedDeviceConnected': 'related device connected',
        }

        self.data = []
        for d in data:
            given_embeddings = torch.stack([self._get_embeddings(entity=t.entity, property=t.property, value=t.value) for t in d['given']], dim=0)
            next_embeddings = [self._get_embeddings(entity=t.entity, property=t.property, value=t.value) for t in d['next']]
            for idx, next in enumerate(next_embeddings):
                if idx == d['answer']:
                    self.data.append((given_embeddings, next, 1))
                else:
                    self.data.append((given_embeddings, next, -1))

        self.given_seq_lengths = torch.LongTensor(list(map(lambda x: len(x[0]), self.data)))
        
        # assert self.given_seq_lengths[0] == len(self.data[0][0])
        # print(self.given_seq_lengths[0])
        # print(len(self.data[0][0]))

        # pad each first element of each sequence to the max length
        MAX_SEQ_LEN = max(self.given_seq_lengths)
        self.data = list(map(lambda x: (torch.cat((x[0], torch.zeros(MAX_SEQ_LEN-len(x[0]), self.EMBEDDING_DIM)), dim=0), x[1], x[2]), self.data))
        self.data = [(x[0], x[1], x[2], y) for x, y in zip(self.data, self.given_seq_lengths)]

    def _get_embeddings(self, entity, property, value):
        assert value in ('True', 'False'), 'value must be True or False'
        if self.embedding_name == 'bpe':
            entity_embedding = torch.tensor(self.embedding.embed(entity).mean(axis=0))
            property_embedding = torch.tensor(self.embedding.embed(self._PROPERTIES_MAP[property]).mean(axis=0))
            # value_embedding = torch.tensor(self.embedding.embed(value).mean(axis=0))
            value_embedding = torch.tensor([1]) if self.embedding.embed(value)=='True' else torch.tensor([0])
        overall_embedding =  torch.cat((entity_embedding, property_embedding, value_embedding), dim=0)
        assert overall_embedding.size()[0] == self.EMBEDDING_DIM, 'embedding size is wrong'
        return overall_embedding

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]



In [ ]:
# construct train, val, test dataset
train_dataset = TechTrackDataset(data['train'])
val_dataset = TechTrackDataset(data['val'])
test_dataset = TechTrackDataset(data['test'])

# construct dataloader
train_dataloader = DataLoader(
                            dataset=train_dataset, 
                            batch_size=args.batch_size, 
                            shuffle=True,
                            )
val_dataloader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=len(val_dataset), shuffle=False)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs200000.model


100%|██████████| 3776868/3776868 [00:01<00:00, 3175697.64B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs200000.d50.w2v.bin.tar.gz


100%|██████████| 38736185/38736185 [00:03<00:00, 10641181.47B/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
len(data['train'][20]['given'])

5

In [ ]:
for d in train_dataloader:
  print(len(d))
  print(d[0].size())
  print(d[1].size())
  print(d[2].size())
  print(d[3].size())
  print(d[2],d[3])
  break

4
torch.Size([32, 30, 101])
torch.Size([32, 101])
torch.Size([32])
torch.Size([32])
tensor([-1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,  1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1]) tensor([ 2,  5,  2,  6,  5,  4,  4,  5,  2,  5, 11,  4,  4,  7,  6,  3,  2,  8,
         4,  4,  5,  3,  1,  1,  2,  3,  4,  7,  3,  6,  4,  1])


In [ ]:
args.embedding_dim = train_dataset.EMBEDDING_DIM

In [ ]:
# build a model
# takes a batch of sequences as input
# uses fasttext embeddings to convert sequences, lstm and mlp
# given a set of candidates, rank them using cosine similarity

class BaselineModel(LightningModule):
    def __init__(self, more_hparams):
        super(BaselineModel, self).__init__()
        assert more_hparams.num_rnn_layers == 1, '1 rnn layer only'
        self.save_hyperparameters()
        self.more_hparams = more_hparams
        # self.embedding = nn.Embedding(self.more_hparams.vocab_size, self.more_hparams.embedding_dim)
        self.lstm = nn.LSTM(input_size = self.more_hparams.embedding_dim, 
                            hidden_size = self.more_hparams.rnn_hidden_dim, 
                            batch_first = True,
                            # proj_size = self.more_hparams.embedding_dim,
                            num_layers = self.more_hparams.num_rnn_layers,
                            dropout = (0 if self.more_hparams.num_rnn_layers == 1 else self.more_hparams.dropout),
                            bidirectional = False,
                            )
        self.mlp = nn.Linear(self.more_hparams.rnn_hidden_dim, self.more_hparams.embedding_dim)
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

        self.CANDIDATE_SET_SIZE = 6

        self.loss='cosine' # 'cosine'

    def forward(self, x):
        given, next, label, seq_len = x

        assert given.size(0) == next.size(0) == label.size(0) == seq_len.size(0), 'batch size mismatch'
        assert given.size(2) == next.size(1), 'embedding dim not same'
        assert given.size(1) == max(seq_len), 'not padded?'

        # x = self.embedding(x)
        ##given = pack_padded_sequence(input=given, lengths=seq_len.cpu().numpy(), batch_first=True, enforce_sorted=False)
        given = pack_padded_sequence(input=given, lengths=seq_len, batch_first=True, enforce_sorted=False)
        outputs, (ht, ct) = self.lstm(given)
        
        ## output, input_sizes = pad_packed_sequence(sequence=output, batch_first=True, )
        ## output = torch.add(output, 1)
        ## assert output.size(1) == 1
        ## output = output.squeeze(1)

        output = ht.squeeze(0)
        assert len(output.size()) == 2, 'output size is wrong (size == 2)'

        output = self.mlp(output)
        assert output.size(0) == seq_len.size(0), 'output size is wrong (dim 0)'
        assert output.size(1) == next.size(1), 'output size is wrong (dim 1'

        if self.loss == 'cosine':
          output = self.cosine_similarity(output, next)
          return output
        else:
          return output, next

    def _compute_loss(self, pred, gold, margin=0):
        # @TODO: implement cosine loss
        # with torch.no_grad():
            
        if self.loss == 'cosine':
            assert ((gold == -1) | (gold == 1)).all()
            return F.margin_ranking_loss(input1=pred, input2=gold, target=gold, margin=margin)
        else:
            return F.mse_loss(pred, gold)

    def _group_queries(self, pred):
        with torch.no_grad():
            out = []
            for i in range(0, len(pred), self.CANDIDATE_SET_SIZE):
                out.append(pred[i:i+self.CANDIDATE_SET_SIZE])
            # print(len(out))
            return torch.stack(out, dim=0)

    def _rank_candidates(self, pred):
        # sort pref along dim 1
        # return indices of sorted pref
        with torch.no_grad():
            return torch.sort(pred, dim=1, descending=True)[1]

    def _precision_of_ranked_candidates(self, ranked_candidates, ans_idx=0, at=1):
        # find P@2, P@3, P@4, ..., P@at
        # return a list of P@i
        with torch.no_grad():
            return torch.mean((ranked_candidates == ans_idx).float(), dim=0)
        

    def training_step(self, batch, batch_idx):
        given, next, label, seq_len = batch
        if self.loss == 'cosine':
          y_hat = self.forward(batch).cpu()
          loss = self._compute_loss(y_hat, label)
        else:
          output, next = self.forward(batch)
          loss = self._compute_loss(output, next)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        given, next, label, seq_len = batch
        if self.loss == 'cosine':
          y_hat = self.forward(batch).cpu()
          loss = self._compute_loss(y_hat, label)
        else:
          output, next = self.forward(batch)
          loss = self._compute_loss(output, next)
          with torch.no_grad():
            y_hat = self.cosine_similarity(output, next)
        self.log("val_loss", loss, logger=True)
        grouped = self._group_queries(y_hat)
        ranked = self._rank_candidates(grouped)
        precisions = self._precision_of_ranked_candidates(ranked, ans_idx=0)
        metrics = {f'val_prec_{i+1}': p for i, p in enumerate(precisions)}
        self.log('val_prec', metrics, logger=True)
        metrics.update({'val_loss': loss, 'loss': loss})
        return metrics

    def test_step(self, batch, batch_idx):
        given, next, label, seq_len = batch
        if self.loss == 'cosine':
          y_hat = self.forward(batch).cpu()
          loss = self._compute_loss(y_hat, label)
        else:
          output, next = self.forward(batch)
          loss = self._compute_loss(output, next)
          with torch.no_grad():
            y_hat = self.cosine_similarity(output, next)
        self.log("test_loss", loss, logger=True)
        grouped = self._group_queries(y_hat)
        ranked = self._rank_candidates(grouped)
        precisions = self._precision_of_ranked_candidates(ranked, ans_idx=0)
        metrics = {f'test_prec_{i+1}': p for i, p in enumerate(precisions)}
        self.log("test_precs", metrics, logger=True)
        metrics.update({'test_loss': loss})
        return metrics

    def configure_optimizers(self):
        optim = AdamW(self.parameters(), lr=self.more_hparams.lr)
        return optim





In [ ]:
#del model;

model = BaselineModel(args)



In [ ]:
seed_everything()

early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=3, verbose=True)
checkpoint_callback = ModelCheckpoint(monitor="val_loss", save_top_k=3, filename="{epoch}-{val_loss:.4f}-{step}")

trainer = Trainer(
    deterministic=True, 
    default_root_dir='out/',
    #gpus=args.gpu,
    max_epochs=100, # args.epochs,
    callbacks=[early_stop_callback, checkpoint_callback]
    # callbacks=[checkpoint_callback]
)


trainer.fit(model, val_dataloader, val_dataloader)




GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name              | Type             | Params
-------------------------------------------------------
0 | lstm              | LSTM             | 7.6 K 
1 | mlp               | Linear           | 1.7 K 
2 | cosine_similarity | CosineSimilarity | 0     
-------------------------------------------------------
9.3 K     Trainable params
0         Non-trainable params
9.3 K     Total params
0.037     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.529


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.528


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.528


In [ ]:
trainer.test(model, val_dataloader, verbose=True)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.5283474326133728,
 'test_precs': {'test_prec_1': tensor(0.1895),
                'test_prec_2': tensor(0.1961),
                'test_prec_3': tensor(0.1765),
                'test_prec_4': tensor(0.2157),
                'test_prec_5': tensor(0.1503),
                'test_prec_6': tensor(0.0719)}}
--------------------------------------------------------------------------------


[{'test_loss': 0.5283474326133728,
  'test_precs': {'test_prec_1': tensor(0.1895),
   'test_prec_2': tensor(0.1961),
   'test_prec_3': tensor(0.1765),
   'test_prec_4': tensor(0.2157),
   'test_prec_5': tensor(0.1503),
   'test_prec_6': tensor(0.0719)}}]